Original Replication Script for Hemauer, Saunders, and Desmarais

Note: This file does not include any gridsearch or hyperparameter tuning. This is just a basic inference replication script.

Last updated: 06/10/2025

In [4]:
### Preprocessing

import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
import random
import time
import warnings

warnings.filterwarnings('ignore')

random.seed(1337)

In [ ]:
### Boehmke et al. 2017 Replication
# Coef estimates are exact, constant estimate is not.

# Data
boehmke_2017_full = pd.read_stata(r"data/boehmke2017.dta")

covariates = ["srcs_decay","nbrs_lag","rpcpinc","totpop","legp_squire",
                "citi6010","unif_rep","unif_dem","time","time_sq","time_cube"]
boehmke_2017 = boehmke_2017_full[["state", "year", "statepol", "adopt"] + covariates].dropna()

# Define X and y
X = boehmke_2017.drop(columns = ['adopt', 'year', 'statepol']).copy()
X = pd.get_dummies(X, columns = ['state'], drop_first = True)  # drop_first avoids perfect multicollinearity
X = sm.add_constant(X)
y = boehmke_2017['adopt']

# Fit Logistic Regression model
start_time = time.time()
logistic = sm.Logit(y.astype(float), X.astype(float)).fit(cov_type = "cluster", cov_kwds = {'groups': boehmke_2017['statepol']})
end_time = time.time()

print(f"Logistic Regression took {end_time - start_time:.2f} seconds")

# Extract summary table
summary_df = logistic.summary2().tables[1]

# Filter out state dummy variables
summary_filtered = summary_df[~summary_df.index.str.startswith("state_")]

print(summary_filtered)

Optimization terminated successfully.
         Current function value: 0.181244
         Iterations 8
Logistic Regression took 0.09 seconds
                Coef.  Std.Err.          z         P>|z|    [0.025    0.975]
const       -4.932003  0.379749 -12.987549  1.439637e-38 -5.676296 -4.187709
srcs_decay   8.526663  0.438523  19.444071  3.271493e-84  7.667175  9.386151
nbrs_lag     0.392840  0.022265  17.643892  1.133760e-69  0.349202  0.436479
rpcpinc      0.573760  0.074898   7.660526  1.851727e-14  0.426962  0.720558
totpop       0.090543  0.028298   3.199597  1.376197e-03  0.035080  0.146007
legp_squire -1.088974  0.687671  -1.583569  1.132918e-01 -2.436784  0.258836
citi6010     0.009835  0.003520   2.793907  5.207549e-03  0.002936  0.016734
unif_rep    -0.020446  0.076089  -0.268708  7.881541e-01 -0.169578  0.128687
unif_dem     0.062910  0.066440   0.946871  3.437047e-01 -0.067310  0.193131
time        -0.135390  0.017594  -7.695380  1.410739e-14 -0.169872 -0.100907
time_sq      

In [2]:
### Boushey 2016 Replication (Table 2: Model 2)
# Coef and constant estimates are exact.

# Data
boushey_2016_full = pd.read_stata(r"data/boushey2016.dta")

# Covariates
covariates = ["policycongruent","gub_election","elect2", "hvd_4yr", "fedcrime",
                "leg_dem_per_2pty","dem_governor","insession","propneighpol",
                "citidist","squire_prof86","citi6008","crimespendpc","crimespendpcsq",
                "violentthousand","pctwhite","stateincpercap","logpop","counter","counter2","counter3"]
boushey_2016 = boushey_2016_full[["state", "styear", "dvadopt"] + covariates].dropna()

# Define X and y
X = boushey_2016[covariates].copy()
X = sm.add_constant(X)
y = boushey_2016['dvadopt']

# Fit Logistic Regression model
start_time = time.time()
logistic = sm.Logit(y.astype(float), X.astype(float)).fit(cov_type = "cluster", cov_kwds = {'groups': boushey_2016['styear']})
end_time = time.time()

print(f"Logistic Regression took {end_time - start_time:.2f} seconds")

# Extract summary table
summary_df = logistic.summary2().tables[1]

# Filter out state dummy variables
summary_filtered = summary_df[~summary_df.index.str.startswith("state_")]

print(summary_filtered)

Optimization terminated successfully.
         Current function value: 0.222943
         Iterations 8
Logistic Regression took 0.02 seconds
                     Coef.  Std.Err.          z          P>|z|    [0.025  \
const            -6.264146  1.150959  -5.442547   5.252402e-08 -8.519984   
policycongruent   0.363093  0.057571   6.306875   2.847255e-10  0.250256   
gub_election     -0.022066  0.090321  -0.244305   8.069945e-01 -0.199092   
elect2            0.032447  0.076580   0.423706   6.717805e-01 -0.117647   
hvd_4yr           0.002792  0.004137   0.674931   4.997198e-01 -0.005316   
fedcrime          1.223034  0.396732   3.082770   2.050833e-03  0.445454   
leg_dem_per_2pty  0.005372  0.002913   1.844528   6.510622e-02 -0.000336   
dem_governor      0.052652  0.068702   0.766377   4.434519e-01 -0.082002   
insession         1.687741  0.227934   7.404505   1.316410e-13  1.240998   
propneighpol      2.359892  0.098142  24.045647  9.270677e-128  2.167537   
citidist         -0.0416

In [ ]:
### Bricker & Lacombe 2021 (Table 3: Model 3)(Monadic Model)

# This works in Stata, mixed effects logit doesn't really exist in Python...

 #melogit adoption std_score initiative init_sigs ///
 #std_pop std_citideology unified std_income std_legp_squire ///
 #duration  durationsq durationcb i.year || policyno:
 #estimates store m1

# Data
bricker_lacombe_2021_full = pd.read_stata(r"data/bricker_lacombe2021.dta")

# Covariates
covariates = ["std_score","initiative","init_sigs","std_population",
                "std_citideology","unified","std_income","std_legp_squire",
                "duration","durationsq","durationcb"]
bricker_lacombe_2021 = bricker_lacombe_2021_full[["state", "year", "policy", "adoption"] + covariates].dropna()

# Define X and y
X = pd.get_dummies(bricker_lacombe_2021, columns = ['year'], drop_first = True)  # drop_first avoids perfect multicollinearity
y = bricker_lacombe_2021['adoption']

# Replace periods with underscores in all column names
X.columns = X.columns.str.replace('.', '_', regex = False)

# Combine X and y into a single DataFrame
data = X.copy()
data['adoption'] = y

# Make sure your outcome is binary
data['adoption'] = data['adoption'].astype(int)

# Fixed effects: all covariates + year dummies
year_cols = [col for col in X.columns if col.startswith('year_')]
exog = sm.add_constant(data[covariates + year_cols])  # add intercept

# Random effects: random intercept for each policy (policyno)
groups = data['policy'].astype('category').cat.codes  # convert to integer codes
exog_re = np.ones((len(data), 1))  # random intercept

# Fit BinomialBayesMixedGLM (logit mixed model)
model = sm.BinomialBayesMixedGLM(data['adoption'], exog, exog_re, groups)
result = model.fit_vb()  # Variational Bayes fit

print(result.summary())

ValueError: len(ident) should match the number of columns of exog_vc

In [13]:
print(formula)

adoption ~ std_score + initiative + init_sigs + std_population + std_citideology + unified + std_income + std_legp_squire + duration + durationsq + durationcb + year_1991_0 + year_1992_0 + year_1993_0 + year_1994_0 + year_1995_0 + year_1996_0 + year_1997_0 + year_1998_0 + year_1999_0 + year_2000_0 + year_2001_0 + year_2002_0 + year_2003_0 + year_2004_0 + year_2005_0 + year_2006_0 + year_2007_0 + year_2008_0 + year_2009_0 + year_2010_0 + year_2011_0 + year_2012_0 + year_2013_0
